In [31]:
# disable future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# safe imports
import os, sys       # arcgis comes with these
import datetime      # arcgis comes with this
import numpy as np   # arcgis comes with this
import pandas as pd  # arcgis comes with this
import matplotlib.pyplot as plt

# risk imports (non-native to arcgis)
try:
    import xarray as xr  # not in arcgis
except:
    arcpy.AddError('Python library Xarray is not installed.')
    raise

# import tools
try:
    # shared folder
    sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\shared')
    sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared')
    import arc, satfetcher, tools  

    # module folder
    sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\modules')
    sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules')
    import ensemble, cog  

except:
    arcpy.AddError('Could not find tenement tools python scripts (modules, shared).')
    raise

In [16]:
# grab parameter values 
#in_lyrs = parameters[0].value      # ncs, vars, types, a, bc, d
in_nodataval = np.nan
in_resample_to = 'highest'

# WORKING ! DELETE !
in_lyrs = [
    ['C:\\Users\\262272G\\Desktop\\test\\phenos_num.nc', 'pos_values', 'Belief', '0.054', 'N/A', '0.442'],
    ['C:\\Users\\262272G\\Desktop\\test\\vegfrax.nc', 'class_1', 'Disbelief', '0.0', 'N/A', '1.0']
]

In [17]:
# check if at least one belief and disbelief layer, else error
ensemble.check_belief_disbelief_exist(in_lyrs)

In [18]:
# clean up stringified a, bc, d numerics
in_lyrs = arc.convert_ensemble_parameters(in_lyrs)

In [19]:
# prepare and load datasets
for lyr in in_lyrs:
    
    # prepare dataset (load depending on type)
    ds = ensemble.prepare_data(file_list=lyr[0],
                               var=lyr[1],
                               nodataval=in_nodataval)
    
    # append if exist, None if not
    lyr.append(ds) if ds else lyr.append(None)

Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: C:\Users\262272G\Desktop\test\phenos_num.nc
Netcdf converted to xarray dataset successfully.
Converting netcdf to an xarray dataset.
Converted netcdf to xarray dataset: C:\Users\262272G\Desktop\test\vegfrax.nc
Netcdf converted to xarray dataset successfully.


In [20]:
# loop each input item and apply sigmoidal automatically
in_lyrs = ensemble.apply_auto_sigmoids(items=in_lyrs)

In [21]:
# export sigmoidals to requested output folder
#ensemble.export_sigmoids(items=in_list, out_path=out_path) 

In [22]:
# seperate into belief, disbelief lists
belief_list, disbelief_list = ensemble.seperate_ds_belief_disbelief(in_lyrs)

In [23]:
# append dempster type label to belief xr datasets
belief_list = ensemble.append_dempster_attr(ds_list=belief_list, 
                                            dempster_label='belief')

# append dempster type label to disbelief xr datasets
disbelief_list = ensemble.append_dempster_attr(ds_list=disbelief_list, 
                                               dempster_label='disbelief')

In [24]:
# resample all datasets to lowest resolution
ds_list = ensemble.resample_datasets(ds_list=belief_list + disbelief_list, 
                                     resample_to=in_resample_to, 
                                     resampling='nearest')

Resampling datasets to highest resolution.


In [25]:
# perform dempster shafer modelling
ds_dempster = ensemble.perform_dempster(ds_list=ds_list)

# compute!
ds_dempster = ds_dempster.compute()